<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/hybrid_qnn_training_dtype_fixed_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pennylane

In [ ]:
#!/usr/bin/env python3
"""
hybrid_qnn_training_dtype_fixed.py

End-to-end hybrid quantum-classical model in PennyLane + PyTorch,
with dtype alignment to avoid Double vs Float errors.

– 4-qubit variational circuit (PennyLane QNode)
– Classical linear readout + sigmoid for parity (XOR) on 4 bits
– Automatic casting of QNode outputs to float32
– Training loop with Adam + BCELoss
– Validation accuracy reporting
"""

import pennylane as qml
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm

# -----------------------------------------------------------------------------
# 1. Define the Hybrid Quantum-Classical Model
# -----------------------------------------------------------------------------
class HybridQuantumNet(nn.Module):
    def __init__(self, n_qubits: int = 4, lr: float = 0.01):
        super().__init__()
        self.n_qubits = n_qubits

        # PennyLane device
        self.dev = qml.device("default.qubit", wires=self.n_qubits)

        # Define variational circuit
        def circuit(inputs):
            for i, angle in enumerate(inputs):
                qml.RY(angle, wires=i)
            return [qml.expval(qml.PauliZ(w)) for w in range(self.n_qubits)]

        # Torch-compatible QNode
        self.qnode = qml.QNode(circuit, self.dev, interface="torch")

        # Classical readout: n_qubits → 1 logit
        self.readout = nn.Linear(self.n_qubits, 1)

        # Optimizer
        self.optimizer = optim.Adam(self.parameters(), lr=lr)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        x: (batch_size, n_qubits) angles in float32
        returns: (batch_size,) probabilities in float32
        """
        q_out = []
        for sample in x:
            expvals = self.qnode(sample)  # returns Python list or torch.Tensor

            # turn list of scalars → tensor, then cast to float32
            if not isinstance(expvals, torch.Tensor):
                expvals = torch.stack(expvals)
            expvals = expvals.to(x.dtype)

            q_out.append(expvals)

        # (batch_size, n_qubits) in float32
        q_out = torch.stack(q_out, dim=0)

        # Linear readout + sigmoid → (batch_size,)
        logits = self.readout(q_out).squeeze(-1)
        return torch.sigmoid(logits)


# -----------------------------------------------------------------------------
# 2. Create Synthetic Parity Dataset
# -----------------------------------------------------------------------------
def make_parity_dataset(n_samples=1000, n_qubits=4, seed=42):
    torch.manual_seed(seed)
    bits   = torch.randint(0, 2, (n_samples, n_qubits)).float()
    angles = bits * torch.pi
    labels = (bits.sum(dim=1) % 2).float()
    return angles, labels


# -----------------------------------------------------------------------------
# 3. Training & Evaluation
# -----------------------------------------------------------------------------
def train_epoch(model, loader, criterion):
    model.train()
    total_loss = 0.0
    for Xb, yb in tqdm(loader, desc="Training", leave=False):
        model.optimizer.zero_grad()
        preds = model(Xb)
        loss  = criterion(preds, yb)
        loss.backward()
        model.optimizer.step()
        total_loss += loss.item() * Xb.size(0)
    return total_loss / len(loader.dataset)


def eval_epoch(model, loader):
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for Xb, yb in loader:
            preds     = model(Xb)
            predicted = (preds >= 0.5).float()
            correct  += (predicted == yb).sum().item()
            total    += Xb.size(0)
    return correct / total


# -----------------------------------------------------------------------------
# 4. Main
# -----------------------------------------------------------------------------
if __name__ == "__main__":
    # Hyperparameters
    N_QUBITS    = 4
    N_SAMPLES   = 1200
    BATCH_SIZE  = 64
    LR          = 0.01
    EPOCHS      = 20
    TRAIN_SPLIT = 1000

    # Prepare data
    angles, labels = make_parity_dataset(N_SAMPLES, N_QUBITS)
    dataset        = TensorDataset(angles, labels)
    train_ds, val_ds = torch.utils.data.random_split(
        dataset, [TRAIN_SPLIT, N_SAMPLES - TRAIN_SPLIT]
    )
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False)

    # Model & loss
    model     = HybridQuantumNet(n_qubits=N_QUBITS, lr=LR)
    criterion = nn.BCELoss()

    # Training loop
    for epoch in range(1, EPOCHS + 1):
        train_loss = train_epoch(model, train_loader, criterion)
        val_acc    = eval_epoch(model, val_loader)
        print(f"Epoch {epoch:2d} | Train Loss: {train_loss:.4f} | Val Acc: {val_acc:.3f}")

    # Final evaluation
    final_acc = eval_epoch(model, val_loader)
    print(f"\nFinal Validation Accuracy: {final_acc:.3f}")